In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib .pyplot as plt

In [4]:
import mysql.connector
from mysql.connector import Error

def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            password = user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection
#Put your MySQL terminal password
password = "Ruby@2020"

#Database name
db = "mysql_python"
connection = create_server_connection("localhost", "root", password)

MySQL Database connection successful


In [8]:
import mysql.connector
from mysql.connector import Error

def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: {err}")
create_database_query = "CREATE DATABASE IF NOT EXISTS mysql_python"
create_database(connection, create_database_query)

Database created successfully


In [14]:
import mysql.connector
from mysql.connector import Error
def create_table(connection, create_table_query):
    cursor = connection.cursor()
    try:
        cursor.execute(create_table_query)
        print("Table created successfully")
    except Error as err:
        print(f"Error: {err}")

def insert_data(connection, insert_data_query, data):
    cursor = connection.cursor()
    try:
        cursor.executemany(insert_data_query, data)
        connection.commit()
        print("Data inserted successfully")
    except Error as err:
        connection.rollback()
        print(f"Error: {err}")

try:
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Ruby@2020",
        database="mysql_python"
    )
    print("Connected to MySQL database")
except Error as err:
    print(f"Error: {err}")


create_table_query = """
CREATE TABLE IF NOT EXISTS students (
    student_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    age INT,
    gender CHAR(1),
    enrollment_date DATE,
    program VARCHAR(50)
)
"""

insert_query = """
INSERT INTO students (name, age, gender, enrollment_date, program) 
VALUES (%s, %s, %s, %s, %s)
"""
data = [
    ('John Doe', 25, 'M', '2022-01-15', 'Computer Science'),
    ('Jane Smith', 22, 'F', '2022-02-20', 'Data Science'),
    ('Michael Johnson', 28, 'M', '2021-12-10', 'Mathematics'),
    ('Emily Brown', 23, 'F', '2022-03-05', 'Engineering'),
    ('David Lee', 27, 'M', '2022-02-10', 'Data Science')
]

if connection.is_connected():
    create_table(connection, create_table_query)
    insert_data(connection, insert_query, data)
else:
    print("Connection to MySQL database failed")
#create_table(connection, create_table_query)
#insert_data(connection, insert_query, data)

Connected to MySQL database
Table created successfully
Data inserted successfully


In [37]:
import mysql.connector
from mysql.connector import Error
from tabulate import tabulate

def display_table_contents(connection, table_name):
    try:
        cursor = connection.cursor()
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        
        if not rows:
            print(f"No records found in the '{table_name}' table.")
            return
            
        column_names = [col[0].title() for col in cursor.description]  # Capitalize column names
        formatted_data = []
        for row in rows:
            formatted_row = [
                row[0], 
                str(row[1]),  # Convert date to string
                str(row[2]),  # Convert age to string
                row[3].upper(), 
                row[4], 
                row[5]
            ]
            formatted_data.append(formatted_row)

        print(tabulate(formatted_data, headers=column_names, tablefmt="grid"))  

    except Error as e:
        print(f"Error: {e}")

# Connect to MySQL database
try:
    connection = mysql.connector.connect(
        host="localhost",
        database="mysql_python",
        user="root",
        password="Ruby@2020"
    )
    print("Connected to MySQL database")
except Error as e:
    print(f"Error: {e}")

# Call the function to display contents of a table
if connection.is_connected():
    display_table_contents(connection, "students")  # Specify your table name here
else:
    print("Connection to MySQL database failed")



Connected to MySQL database
+--------------+-----------------+-------+----------+-------------------+------------------+
|   Student_Id | Name            |   Age | Gender   | Enrollment_Date   | Program          |
+==============+=================+=======+==========+===================+==================+
|            1 | John Doe        |    25 | M        | 2022-01-15        | Computer Science |
+--------------+-----------------+-------+----------+-------------------+------------------+
|            2 | Jane Smith      |    22 | F        | 2022-02-20        | Data Science     |
+--------------+-----------------+-------+----------+-------------------+------------------+
|            3 | Michael Johnson |    28 | M        | 2021-12-10        | Mathematics      |
+--------------+-----------------+-------+----------+-------------------+------------------+
|            4 | Emily Brown     |    23 | F        | 2022-03-05        | Engineering      |
+--------------+-----------------+-------+

In [38]:
import mysql.connector
from mysql.connector import Error

def count_total_students(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("SELECT COUNT(*) AS total_students FROM students")
        result = cursor.fetchone()
        total_students = result[0]
        print(f"Total Students: {total_students}")
    except Error as e:
        print(f"Error: {e}")

# Example usage:
try:
    connection = mysql.connector.connect(
        host="localhost",
        database="mysql_python",
        user="root",
        password="Ruby@2020"
    )
    print("Connected to MySQL database")
    count_total_students(connection)
except Error as e:
    print(f"Error: {e}")
finally:
    if connection.is_connected():
        connection.close()
        print("Disconnected from MySQL database")


Connected to MySQL database
Total Students: 15
Disconnected from MySQL database


In [5]:
import mysql.connector
from mysql.connector import Error

def add_current_date_column(connection, students, column_name="Today's Date"):
    """Adds a new column named 'column_name' with the current date (DATE data type)
       as the default value to the specified table, if it doesn't already exist.

    Args:
        connection (mysql.connector.connection): The connection object to the MySQL database.
        table_name (str): The name of the table to add the column to.
        column_name (str, optional): The desired name for the new column. Defaults to "Today's Date".

    Returns:
        None
    """

    try:
        cursor = connection.cursor()

        # Check if the column already exists (optional)
        check_query = f"DESCRIBE {students}"
        cursor.execute(check_query)
        existing_columns = [col[0] for col in cursor.fetchall()]

        if column_name not in existing_columns:
            query = f"ALTER TABLE {table_name} ADD COLUMN `{column_name}` DATE DEFAULT (CURRENT_DATE())"
            cursor.execute(query)
            print(f"Column '{column_name}' added successfully to table '{students}'.")
        else:
            print(f"Column '{column_name}' already exists in table '{students}'.")

    except Error as err:
        print(f"Error adding column: {err}")

# Example usage with checking for existing column
try:
    connection = mysql.connector.connect(
        host="localhost",
        database="mysql_python",
        user="root",
        password="Ruby@2020"  # Replace with your actual password
    )
    print("Connected to MySQL database")

    add_current_date_column(connection, "students")

except Error as err:
    print(f"Connection error: {err}")

finally:
    if connection.is_connected():
        connection.close()
        print("Disconnected from MySQL database")

Connected to MySQL database
Column 'Today's Date' already exists in table 'students'.
Disconnected from MySQL database
